## Import Libraries

In [1]:
import arcpy 
import pandas as pd 
import numpy as np
import os
import glob as gb

## Set Directory

In [2]:
path = 'C:\\Users\\Thepr\\Documents\\Consulting\\Soft Parcel Analysis'
os.chdir(path)

In [3]:
ProjectGDB =  r"C:\\Users\\Thepr\\Documents\\Consulting\\Soft Parcel Analysis\\Soft Parcel Analysis.gdb"
arcpy.env.overwriteOutput = True
arcpy.env.workspace = ProjectGDB

# Zoning 

- Convert Zoning Districts to Point
- Join the Zoning Districts to the Arlington Parcel Using Spatial Join

In [4]:
if arcpy.Exists('ZoningDistrictsPoints'):
    arcpy.Delete_management('ZoningDistrictsPoints')
    
arcpy.management.FeatureToPoint("ZoningDistricts","ZoningDistrictsPoints","CENTROID")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\ZoningDistrictsPoints'>

In [5]:
Zfields = []
for row in arcpy.ListFields('ZoningDistricts'):
    Zfields.append(row.name)
print(Zfields)

['OBJECTID_1', 'Shape', 'OBJECTID', 'ZONINGDETA', 'DISTRICT', 'PD_RESIDEN', 'PART', 'ZONINGCASE', 'ORDINANCE', 'EFFECTIVED', 'ZONINGDESC', 'SHAPESTAre', 'SHAPESTLen', 'Shape_Length', 'Shape_Area', 'Zoning', 'DensityCompatible']


# Zoning Desgination 

- Create **Zoning Dictionary**
- Add a new field 
- Calculate full zoning names into the new fields

In [6]:
zoning = {'ETJ': 'Extra Territorial Jurisdiction', 'RMU': 'Regional Mixed-Use', 'NMU': 'Neighborhood Mixed-Use', 
          'IM': 'Industrial Manufacturing', 'LI': 'Light Industrial', 'FH': 'Flex Hybrid', 'BP': 'Business Park', 
          'DB': 'Downtown Business', 'HC': 'Highway Commercial', 'GC': 'General Commercial', 'CC': 'Community Commercial', 
          'NC': 'Neighborhood Commercial', 'OC': 'Office Commercial', 'LO': 'Limited Office', 'PD': 'Planned Development',
          'MH': 'Manufactured Housing', 'VG': 'Village on the Green', 'RMF-22': 'Residential Multi-Family 22', 
          'RM-12': 'Residential Medium Density', 'RS-5': 'Residential Single-Family 5', 
          'RS-7.2': 'Residential Single-Family 7.2', 'RS-15': 'Residential Single-Family 15',
          'RS-20': 'Residential Single-Family 20', 'RE': 'Residential Estate', 'UTA': 'UT Arlington',
          'APO': 'Airport Overlay', 'DNO': 'Downtown Neighborhood Overlay', 'EDO': 'Entertainment District Overlay', 
          'CDO': 'Conservation District Overlay', 'LPO': 'Landmark Preservation Overlay', 
          'LCMUO': 'Lamar Collins Mixed-Use Overlay',
          'PD07-5R4-DP14': 'Planned Development (PD07-5R4-DP14)', 'PD07-5R4-DP16': 'Planned Development (PD07-5R4-DP16)',
          'APO-CC': 'Airport Overlay - Community Commercial', 'APO-GC': 'Airport Overlay - General Commercial',
          'APO-IM': 'Airport Overlay - Industrial Manufacturing', 'APO-LI': 'Airport Overlay - Light Industrial',
          'APO-LI-PD': 'Airport Overlay - Light Industrial - Planned Development',
          'APO-NC': 'Airport Overlay - Neighborhood Commercial', 'APO-OC': 'Airport Overlay - Office Commercial', 
          'APO-PD': 'Airport Overlay - Planned Development', 'APO-RE': 'Airport Overlay - Residential Estate',
          'APO-RM-12': 'Airport Overlay - Residential Medium Density', 
          'APO-RMF-22': 'Airport Overlay - Residential Multi-Family 22', 
          'APO-RS-5': 'Airport Overlay - Residential Single-Family 5', 
          'APO-RS-7.2': 'Airport Overlay - Residential Single-Family 7.2', 'BP-PD': 'Business Park - Planned Development',
          'CC-PD-RMF-22': 'Community Commercial - Planned Development - Residential Multi-Family 22',
          'CDO-RS-7.2': 'Conservation District Overlay - Residential Single-Family 7.2', 
          'DNO-CC': 'Downtown Neighborhood Overlay - Community Commercial', 
          'DNO-GC': 'Downtown Neighborhood Overlay - General Commercial', 
          'DNO-LI': 'Downtown Neighborhood Overlay - Light Industrial', 
          'DNO-LO': 'Downtown Neighborhood Overlay - Limited Office',
          'DNO-LPO-GC': 'Downtown Neighborhood Overlay - Landmark Preservation Overlay - General Commercial',
          'DNO-LPO-NC': 'Downtown Neighborhood Overlay - Landmark Preservation Overlay - Neighborhood Commercial',
          'DNO-NC': 'Downtown Neighborhood Overlay - Neighborhood Commercial', 
          'DNO-OC': 'Downtown Neighborhood Overlay - Office Commercial', 
          'DNO-PD': 'Downtown Neighborhood Overlay - Planned Development', 
          'DNO-PD-DB': 'Downtown Neighborhood Overlay - Planned Development - Downtown Business', 
          'DNO-PD-RM-12': 'Downtown Neighborhood Overlay - Planned Development - Residential Medium Density', 
          'DNO-PD-RMF-22': 'Downtown Neighborhood Overlay - Planned Development - Residential Multi-Family 22',
          'DNO-RM-12': 'Downtown Neighborhood Overlay - Residential Medium Density',
          'DNO-RMF-22': 'Downtown Neighborhood Overlay - Residential Multi-Family 22', 
          'DNO-RS-7.2': 'Downtown Neighborhood Overlay - Residential Single-Family 7.2',
          'EDO-CC': 'Entertainment District Overlay - Community Commercial', 
          'EDO-GC': 'Entertainment District Overlay - General Commercial',
          'EDO-IM': 'Entertainment District Overlay - Industrial Manufacturing',
          'EDO-LI': 'Entertainment District Overlay - Light Industrial',
          'EDO-NC': 'Entertainment District Overlay - Neighborhood Commercial', 
          'EDO-OC': 'Entertainment District Overlay - Office Commercial', 
          'EDO-PD': 'Entertainment District Overlay - Planned Development', 
          'EDO-PD-RM-12': 'Entertainment District Overlay - Planned Development - Residential Medium Density', 
          'EDO-RM-12': 'Entertainment District Overlay - Residential Medium Density',
          'EDO-RMF-22': 'Entertainment District Overlay - Residential Multi-Family 22',
          'EDO-RS-7.2': 'Entertainment District Overlay - Residential Single-Family 7.2', 
          'LCMUO-CC': 'Lamar Collins Mixed-Use Overlay - Community Commercial', 
          'LCMUO-GC': 'Lamar Collins Mixed-Use Overlay - General Commercial', 
          'LCMUO-PD': 'Lamar Collins Mixed-Use Overlay - Planned Development',
          'LCMUO-RMF-22': 'Lamar Collins Mixed-Use Overlay - Residential Multi-Family 22',
          'LPO-CC': 'Landmark Preservation Overlay - Community Commercial',
          'LPO-DB': 'Landmark Preservation Overlay - Downtown Business',
          'LPO-NC': 'Landmark Preservation Overlay - Neighborhood Commercial', 
          'LPO-RM-12': 'Landmark Preservation Overlay - Residential Medium Density',
          'LPO-RS-7.2': 'Landmark Preservation Overlay - Residential Single-Family 7.2', 
          'PD EDO-RM-12': 'Planned Development - Entertainment District Overlay - Residential Medium Density', 
          'PD RMF-22': 'Planned Development - Residential Multi-Family 22', 
          'PD-APO-GC': 'Planned Development - Airport Overlay - General Commercial', 
          'PD-CA': 'Planned Development - Civic Area', 'PD-CC': 'Planned Development - Community Commercial', 
          'PD-DB': 'Planned Development - Downtown Business', 
          'PD-DNO-RMF-22': 'Planned Development - Downtown Neighborhood Overlay - Residential Multi-Family 22',
          'PD-EDO-CC': 'Planned Development - Entertainment District Overlay - Community Commercial', 
          'PD-EDO-RMU': 'Planned Development - Entertainment District Overlay - Regional Mixed-Use', 
          'PD-GC': 'Planned Development - General Commercial', 'PD-IM': 'Planned Development - Industrial Manufacturing', 
          'PD-LI': 'Planned Development - Light Industrial', 
          'PD-LI-CC': 'Planned Development - Light Industrial - Community Commercial',
          'PD-LO': 'Planned Development - Limited Office', 'PD-OC': 'Planned Development - Office Commercial',
          'PD-RE': 'Planned Development - Residential Estate', 'PD-RM12': 'Planned Development - Residential Medium Density', 
          'PD-RM-12': 'Planned Development - Residential Medium Density',
          'PD-RM-12-CC': 'Planned Development - Residential Medium Density - Community Commercial', 
          'PD-RM-12-NC': 'Planned Development - Residential Medium Density - Neighborhood Commercial',
          'PD-RM-12-RMF-22': 'Planned Development - Residential Medium Density - Residential Multi-Family 22',
          'PD-RMF22': 'Planned Development - Residential Multi-Family 22', 
          'PD-RMF-22': 'Planned Development - Residential Multi-Family 22',
          'PD-RMF-22-CC': 'Planned Development - Residential Multi-Family 22 - Community Commercial', 
          'PD-RMF-22-GC': 'Planned Development - Residential Multi-Family 22 - General Commercial', 
          'PD-RS-15': 'Planned Development - Residential Single-Family 15',
          'PD-RS-5': 'Planned Development - Residential Single-Family 5', 
          'PD-RS-5-CC': 'Planned Development - Residential Single-Family 5 - Community Commercial', 
          'PD-RS-7.2': 'Planned Development - Residential Single-Family 7.2', 
          'PD-RS-7.2-RM-12-CC': 'Planned Development - Residential Single-Family 7.2 - Residential Medium Density - Community Commercial'}

In [7]:
arcpy.management.AddField("ZoningDistricts","Zoning","TEXT",None,None,1000,"Zoning","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\ZoningDistricts'>

In [8]:
with arcpy.da.UpdateCursor('ZoningDistricts', ['ZONINGDETA', 'Zoning']) as cursor:
    for row in cursor:
        code = row[0]
        if code in zoning:
            row[1] = zoning[code]
        cursor.updateRow(row)

# High Density and Mixed Use Compatible

- Create a list of Zoning Compatible with Mixed Use or High Density
- Add Field 
- Calculate Compatible Zones into those fields

In [9]:
compatiblezones = [
    "RMU", "NMU", "LCMUO", "LCMUO-CC", "LCMUO-GC", "LCMUO-PD", "LCMUO-RMF-22",
    "VG", "DB", "DNO-CC", "DNO-GC", "DNO-OC", "DNO-LO", "DNO-RM-12", "DNO-RMF-22",
    "DNO-PD", "DNO-PD-DB", "DNO-PD-RMF-22", "DNO-PD-RM-12",
    "EDO-CC", "EDO-GC", "EDO-NC", "EDO-OC", "EDO-RM-12", "EDO-RMF-22",
    "PD-EDO-CC", "PD-EDO-RMU", "GC", "CC", "NC", "OC", "LO",
    "RMF-22", "RM-12", "PD-RMF-22", "PD-RM-12", "PD-RMF-22-GC", "PD-RMF-22-CC",
    "PD-RM-12-CC", "PD-RM-12-NC", "PD-RM-12-RMF-22", "CC-PD-RMF-22",
    "PD07-5R4-DP14", "PD07-5R4-DP16", "APO-CC", "APO-GC", "APO-NC", "APO-OC",
    "APO-RM-12", "APO-RMF-22", "APO-LI-PD", "APO-PD", "PD-APO-GC"
]

In [10]:
arcpy.management.AddField("ZoningDistricts","DensityCompatible","TEXT",None,None,1000,"DensityCompatible","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\ZoningDistricts'>

In [11]:
with arcpy.da.UpdateCursor('ZoningDistricts', ['ZONINGDETA', "DensityCompatible"]) as cursor:
    for row in cursor:
        zoning = row[0]
        if zoning in compatiblezones:
            row[1] = "Yes"
        else:
            row[1] = "No"
        cursor.updateRow(row)

- **Join Zoning to Landuse** 

In [12]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable('ArlingtonParcelLandUse')
fieldmapping.addTable('ZoningDistricts')

analysisfields = ['TAXPIN', 'ACRES', 'CALCULATED_ACREAGE',  'RP', 'Situs_Addr','LegalDescr',  'County', 'City', 
                  'Land_Value', 'Improvemen', 'Total_Valu', 'Year_Built', 'Land_Acres','Land_SqFt', 
                  'Appraised_', 'TaxParcelI','LandCode','LandCode_1','LandCode_2','LandCodeDe',
                  'ChangeFlag','ZONINGDETA','DISTRICT','ZONINGDESC','Zoning','DensityCompatible']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('ArlingtonParcelLanduseZoning'):
    arcpy.Delete_management('ArlingtonParcelLanduseZoning')
        
Parcels_Zoning = arcpy.analysis.SpatialJoin("ArlingtonParcelLandUse","ZoningDistricts","ArlingtonParcelLanduseZoning",
                                        "JOIN_ONE_TO_ONE","KEEP_ALL",fieldmapping,"LARGEST_OVERLAP",None,"",None)


# Parcels

- Land use

In [13]:
LUfields = []
for row in arcpy.ListFields('ArlingtonParcelLanduseZoning'):
    LUfields.append(row.name)
print(LUfields)

['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'TaxParcelI', 'LandCode', 'LandCodeDe', 'LandCode_1', 'LandCode_2', 'ChangeFlag', 'ZONINGDETA', 'DISTRICT', 'ZONINGDESC', 'Zoning', 'DensityCompatible', 'Shape_Length', 'Shape_Area']


In [14]:
parcelfields = []
for row in arcpy.ListFields('TADParcels2024'):
    parcelfields.append(row.name)
print(parcelfields)

['OBJECTID_1', 'Shape', 'TAXPIN', 'ACRES', 'PARCELTYPE', 'CALCULATED', 'created_da', 'last_edi_1', 'Shape_STAr', 'Shape_STLe', 'OBJECTID', 'RP', 'Appraisal_Year', 'Account_Number', 'Record_Type', 'Sequence_No', 'PIDN', 'Owner_Name', 'Owner_Address', 'Owner_CityState', 'Owner_Zip', 'Owner_Zip4', 'Owner_CRRT', 'Situs_Address', 'Property_Class', 'TAD_Map', 'MAPSCO', 'Exemption_Code', 'State_Use_Code', 'LegalDescription', 'Notice_Date', 'County', 'City', 'School', 'Num_Special_Dist', 'Spec1', 'Spec2', 'Spec3', 'Spec4', 'Spec5', 'Deed_Date', 'Deed_Book', 'Deed_Page', 'Land_Value', 'Improvement_Value', 'Total_Value', 'Garage_Capacity', 'Num_Bedrooms', 'Num_Bathrooms', 'Year_Built', 'Living_Area', 'Swimming_Pool_Ind', 'ARB_Indicator', 'Ag_Code', 'Land_Acres', 'Land_SqFt', 'Ag_Acres', 'Ag_Value', 'Central_Heat_Ind', 'Central_Air_Ind', 'Structure_Count', 'From_Accts', 'Appraisal_Date', 'Appraised_Value', 'GIS_Link', 'Instrument_No', 'Overlap_Flag', 'Shape_Length_1', 'Shape_Area_1', 'Shape_Lengt

- Convert Parcel to Points
- Spatially Join Parcels to Arlington Parcels Landuse

In [15]:
if arcpy.Exists('TADParcelsLandUsePoints2023'):
    arcpy.Delete_management('TADParcelsLandUsePoints2023')

arcpy.management.FeatureToPoint("TADParcels2023","TADParcelsLandUsePoints2023","CENTROID")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcelsLandUsePoints2023'>

In [16]:
if arcpy.Exists('TADParcelsLandUsePoints2024'):
    arcpy.Delete_management('TADParcelsLandUsePoints2024')
    
arcpy.management.FeatureToPoint("TADParcels2024","TADParcelsLandUsePoints2024","CENTROID")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcelsLandUsePoints2024'>

##  **2023**

- Spatial Join for all 2023 Parcels
- Select only fields needed for the analysis

In [17]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable('ArlingtonParcelLanduseZoning')
fieldmapping.addTable('TADParcelsLandUsePoints2023')

analysisfields = ['TAXPIN', 'ACRES', 'CALCULATED_ACREAGE',  'RP', 'Situs_Addr','LegalDescr',  'County', 'City', 
                  'Land_Value', 'Improvemen', 'Total_Valu', 'Year_Built', 'Land_Acres','Land_SqFt', 
                  'Appraised_', 'TaxParcelI','LandCode','LandCode_1','LandCode_2','LandCodeDe',
                  'ChangeFlag','ZONINGDETA','DISTRICT','ZONINGDESC','Zoning','DensityCompatible']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('TADParcel2023LandUse'):
    arcpy.Delete_management('TADParcel2023LandUse')
        
Parcels_LU = arcpy.analysis.SpatialJoin("ArlingtonParcelLanduseZoning","TADParcelsLandUsePoints2023","TADParcel2023LandUse",
                                        "JOIN_ONE_TO_ONE","KEEP_ALL",fieldmapping,"CONTAINS",None,"",None)



## Field Calculations

### 2023

- Add fields 
- Convert Improvement and Total value to Double
- Compute improvement to land ratio
- Calculate Geometry

- **Year Built**

In [18]:
arcpy.management.AddField("TADParcel2023LandUse","Year","DOUBLE",None,None,None,"Year","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

In [19]:
arcpy.management.CalculateField(
    "TADParcel2023LandUse",
    "Year",
    "cal(!Year_Built!)",
    expression_type="PYTHON3",
    code_block="""
def cal(val):
    try:
        return int(val.strip())
    except:
        return None
"""
)


<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

- **Improvement Value**

In [20]:
arcpy.management.AddField("TADParcel2023LandUse","IMPvalue","DOUBLE",None,None,None,"IMPvalue","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

In [21]:
arcpy.management.CalculateField("TADParcel2023LandUse","IMPvalue","!Improvemen!","PYTHON3","","TEXT","NO_ENFORCE_DOMAINS")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

- **Land Value**

In [22]:
arcpy.management.AddField("TADParcel2023LandUse","LandValue","DOUBLE",None,None,None,"LandValue","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

In [23]:
arcpy.management.CalculateField("TADParcel2023LandUse","LandValue","!Land_Value!","PYTHON3","","TEXT","NO_ENFORCE_DOMAINS")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

- **Improvement Ratio**

In [24]:
arcpy.management.AddField("TADParcel2023LandUse","ILR","DOUBLE",None,None,None,"ILR","NULLABLE",
                          "NON_REQUIRED","")


<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

In [25]:
arcpy.management.CalculateField("TADParcel2023LandUse","ILR","calc(!IMPvalue!, !LandValue!)","PYTHON3",
    """
def calc(imp, land):
    try:
        if imp is None or land is None or land == 0:
            return None
        return round(imp / land, 2)
    except:
        return None
""",
    "DOUBLE",
    "NO_ENFORCE_DOMAINS"
)

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

- **Acreage**

In [26]:
arcpy.management.AddField("TADParcel2023LandUse","Acreage","DOUBLE",None,None,None,"Acreage","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

In [27]:
arcpy.management.CalculateGeometryAttributes("TADParcel2023LandUse","Acreage AREA","","ACRES",
                                             arcpy.SpatialReference(102738),"SAME_AS_INPUT")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2023LandUse'>

# Selection Criteria 2023

- ILR is less than or equal to 0.5
- Landuse is Vacant or Parking 
- Zoning Designation is not Conservation District Overlay or Planned Development 
- Year Built is 0 or Year Built is on or Before 2005

In [28]:
Softparcels2023 = arcpy.management.SelectLayerByAttribute(
    "TADParcel2023LandUse",
    "NEW_SELECTION",
    (
        "ILR <= 0.5 AND "
        "LandCode_2 IN ("
            "'Vacant - Under Development (14)', "
            "'Vacant-Developable (2)', "
            "'Parking (13)'"
        ") AND "
        "Zoning NOT IN ("
            "'Conservation District Overlay - Residential Single-Family 7.2', "
            "'Planned Development', "
            "'Planned Development - Airport Overlay - General Commercial', "
            "'Planned Development - Civic Area', "
            "'Planned Development - Community Commercial', "
            "'Planned Development - Entertainment District Overlay - Community Commercial', "
            "'Planned Development - Entertainment District Overlay - Regional Mixed-Use', "
            "'Planned Development - General Commercial', "
            "'Planned Development - Industrial Manufacturing', "
            "'Planned Development - Light Industrial', "
            "'Planned Development - Office Commercial', "
            "'Planned Development - Residential Medium Density', "
            "'Planned Development - Residential Medium Density - Community Commercial', "
            "'Planned Development - Residential Medium Density - Neighborhood Commercial', "
            "'Planned Development - Residential Medium Density - Residential Multi-Family 22', "
            "'Planned Development - Residential Multi-Family 22', "
            "'Planned Development - Residential Multi-Family 22 - Community Commercial', "
            "'Planned Development - Residential Multi-Family 22 - General Commercial', "
            "'Planned Development - Residential Single-Family 15', "
            "'Planned Development - Residential Single-Family 5', "
            "'Planned Development - Residential Single-Family 5 - Community Commercial', "
            "'Planned Development - Residential Single-Family 7.2', "
            "'Planned Development - Residential Single-Family 7.2 - Residential Medium Density - Community Commercial', "
            "'Planned Development (PD07-5R4-DP14)', "
            "'Planned Development (PD07-5R4-DP16)'"
        ") AND "
        "(Year <= 2005) AND "
        "DensityCompatible = 'Yes'"
    ),
    None
)


In [29]:
# Softparcels2023 = arcpy.management.SelectLayerByAttribute(
#     "TADParcel2023LandUse",
#     "NEW_SELECTION",
#     (
#         "ILR <= 0.5 AND "
#         "LandCode_2 IN ("
#             "'Vacant - Under Development (14)', "
#             "'Vacant-Developable (2)', "
#             "'Parking (13)'"
#         ") AND "
#         "Zoning NOT IN ("
#             "'Conservation District Overlay - Residential Single-Family 7.2'"
#         ") AND "
#         "(Year <= 2005) AND "
#         "DensityCompatible = 'Yes'"
#     ),
#     None
# )


In [30]:
softparcels = int(arcpy.management.GetCount(Softparcels2023)[0])
print(softparcels)

525


In [31]:
# fieldmapping = arcpy.FieldMappings()
# fieldmapping.addTable(Softparcels2023)

# if arcpy.Exists('TADSoftParcels2023'):
#     arcpy.Delete_management('TADSoftParcels2023')


# arcpy.conversion.ExportFeatures(Softparcels2023,"TADSoftParcels2023","","NOT_USE_ALIAS",fieldmapping,None)
        

In [32]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(Softparcels2023)

analysisfields = ['TAXPIN', 'RP', 'Land_Value', 'Improvemen', 'Total_Valu', 'Year',
                  'LandCode_2','LandCodeDe','ZONINGDETA','Acreage','IMPvalue','LandValue','ILR',
                  'DISTRICT','ZONINGDESC','Zoning','DensityCompatible']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

if arcpy.Exists('TADSoftParcels2023'):
    arcpy.Delete_management('TADSoftParcels2023')

arcpy.conversion.ExportFeatures(Softparcels2023,"TADSoftParcels2023","","NOT_USE_ALIAS",fieldmapping,None)
        

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcels2023'>

In [33]:
recordscount = int(arcpy.management.GetCount("TADSoftParcels2023")[0])
print(recordscount)

525


# Clusters 2023



- Create Clusters
- Calculate acreage
- Filter for clusters less than 50 acres

In [34]:
if arcpy.Exists('TADSoftParcelsClusters2023'):
    arcpy.Delete_management('TADSoftParcelsClusters2023')
    
if arcpy.Exists('TADSoftParcelsClustersTable2023'):
    arcpy.Delete_management('TADSoftParcelsClustersTable2023')


arcpy.cartography.AggregatePolygons("TADSoftParcels2023",
                                    'TADSoftParcelsClusters2023',
                                    "500 Feet",
                                    "0 SquareFeetUS",
                                    "0 SquareFeetUS",
                                    "NON_ORTHOGONAL",
                                    None,
                                    'TADSoftParcelsClustersTable2023'
                                    ,None)


<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcelsClusters2023'>

In [35]:
recordscount = int(arcpy.management.GetCount("TADSoftParcelsClusters2023")[0])
print(recordscount)

200


In [36]:
arcpy.management.AddField("TADSoftParcelsClusters2023","clusterAcreage","DOUBLE",None,None,None,"clusterAcreage","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcelsClusters2023'>

In [37]:
arcpy.management.CalculateGeometryAttributes("TADSoftParcelsClusters2023","clusterAcreage AREA","","ACRES",
                                             arcpy.SpatialReference(102738),"SAME_AS_INPUT")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcelsClusters2023'>

In [38]:
if arcpy.Exists('SoftParcels2023Circles'):
    arcpy.Delete_management('SoftParcels2023Circles')

arcpy.management.MinimumBoundingGeometry("TADSoftParcelsClusters2023",
                                         "SoftParcels2023Circles",
                                         "CIRCLE",
                                         "NONE",
                                         None,
                                         "NO_MBG_FIELDS")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\SoftParcels2023Circles'>

In [39]:
recordscount = int(arcpy.management.GetCount("SoftParcels2023Circles")[0])
print(recordscount)

200


# Density Based Clustering 2023
- Convert SoftParcels to Points
- Run DBSCAN On Points 
- Adjust DBSCAN for 1000 feet

In [40]:
if arcpy.Exists('TADSoftParcelsPoints2023'):
    arcpy.Delete_management('TADSoftParcelsPoints2023')

arcpy.management.FeatureToPoint("TADSoftParcels2023","TADSoftParcelsPoints2023","CENTROID")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcelsPoints2023'>

In [41]:
if arcpy.Exists('TADSoftparcelsDensityBasedClusterDistance2023'):
    arcpy.Delete_management('TADSoftparcelsDensityBasedClusterDistance2023')

arcpy.stats.DensityBasedClustering("TADSoftParcelsPoints2023",
                                   "TADSoftparcelsDensityBasedClusterDistance2023",
                                   "DBSCAN",
                                   2,
                                   "500 Feet",
                                   None,
                                   None,
                                   None)

recordscount = int(arcpy.management.GetCount("TADSoftparcelsDensityBasedClusterDistance2023")[0])
print(recordscount)

525


In [42]:
clustersDB2023 = arcpy.management.SelectLayerByAttribute("TADSoftparcelsDensityBasedClusterDistance2023",
                                        "NEW_SELECTION",
                                        "COLOR_ID > 0",
                                        None)

recordscount = int(arcpy.management.GetCount(clustersDB2023)[0])
print(recordscount)

360


In [43]:
if arcpy.Exists('SoftParcelsPreferred2023'):
    arcpy.Delete_management('SoftParcelsPreferred2023')

arcpy.analysis.SpatialJoin("TADSoftParcels2023",clustersDB2023,"SoftParcelsPreferred2023",
                                        "JOIN_ONE_TO_ONE","KEEP_COMMON",'',"COMPLETELY_CONTAINS",None,"",None)
recordscount = int(arcpy.management.GetCount("SoftParcelsPreferred2023")[0])
print(recordscount)

360


In [44]:
if arcpy.Exists('SoftParcelsBoundary2023'):
    arcpy.Delete_management('SoftParcelsBoundary2023')

arcpy.management.MinimumBoundingGeometry("SoftParcelsPreferred2023","SoftParcelsBoundary2023","CIRCLE","LIST",
                                         "CLUSTER_ID;COLOR_ID","NO_MBG_FIELDS")

recordscount = int(arcpy.management.GetCount("SoftParcelsBoundary2023")[0])
print(recordscount)

99


## **2024**

- Spatial Join for all 2024 Parcels
- Select only fields needed for the analysis

In [45]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable('ArlingtonParcelLanduseZoning')
fieldmapping.addTable('TADParcelsLandUsePoints2024')

analysisfields = ['TAXPIN', 'ACRES', 'CALCULATED',  'RP', 'Situs_Addr','LegalDescr',  'County', 'City', 
                  'Land_Value', 'Improvement_Value', 'Total_Value', 'Year_Built', 'Land_Acres','Land_SqFt', 
                  'Appraised_', 'TaxParcelI','LandCode','LandCode_1','LandCode_2','LandCodeDe',
                  'ChangeFlag','ZONINGDETA','DISTRICT','ZONINGDESC','Zoning','DensityCompatible']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        
if arcpy.Exists('TADParcel2024LandUse'):
    arcpy.Delete_management('TADParcel2024LandUse')
        
Parcels_LU = arcpy.analysis.SpatialJoin("ArlingtonParcelLanduseZoning","TADParcelsLandUsePoints2024","TADParcel2024LandUse",
                                        "JOIN_ONE_TO_ONE","KEEP_ALL",fieldmapping,"CONTAINS",None,"",None)



## Field Calculations

### 2024

- Add fields 
- Convert Improvement and Total value to Double
- Compute improvement to land ratio
- Calculate Geometry

- **Year Built**

In [46]:
arcpy.management.AddField("TADParcel2024LandUse","Year","DOUBLE",None,None,None,"Year","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

In [47]:
arcpy.management.CalculateField(
    "TADParcel2024LandUse",
    "Year",
    "cal(!Year_Built!)",
    expression_type="PYTHON3",
    code_block="""
def cal(val):
    try:
        return int(val)
    except:
        return None
"""
)



<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

- **Improvement Value**

In [48]:
arcpy.management.AddField("TADParcel2024LandUse","IMPvalue","DOUBLE",None,None,None,"IMPvalue","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

In [49]:
arcpy.management.CalculateField("TADParcel2024LandUse","IMPvalue","!Improvement_Value!","PYTHON3","","TEXT",
                                "NO_ENFORCE_DOMAINS")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

- **Land Value**

In [50]:
arcpy.management.AddField("TADParcel2024LandUse","LandValue","DOUBLE",None,None,None,"LandValue","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

In [51]:
arcpy.management.CalculateField("TADParcel2024LandUse","LandValue","!Land_Value!","PYTHON3","","TEXT",
                                "NO_ENFORCE_DOMAINS")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

- **Improvement Ratio**

In [52]:
arcpy.management.AddField("TADParcel2024LandUse","ILR","DOUBLE",None,None,None,"ILR","NULLABLE",
                          "NON_REQUIRED","")


<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

In [53]:
arcpy.management.CalculateField("TADParcel2024LandUse","ILR","calc(!IMPvalue!, !LandValue!)","PYTHON3",
    """
def calc(imp, land):
    try:
        if imp is None or land is None or land == 0:
            return None
        return round(imp / land, 2)
    except:
        return None
""",
    "DOUBLE",
    "NO_ENFORCE_DOMAINS"
)

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

- **Acreage**

In [54]:
arcpy.management.AddField("TADParcel2024LandUse","Acreage","DOUBLE",None,None,None,"Acreage","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

In [55]:
arcpy.management.CalculateGeometryAttributes("TADParcel2024LandUse","Acreage AREA","","ACRES",
                                             arcpy.SpatialReference(102738),"SAME_AS_INPUT")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADParcel2024LandUse'>

# Selection Criteria 2024

- ILR is less than or equal to 0.5
- Landuse is Vacant or Parking 
- Zoning Designation is not Conservation District Overlay or Planned Development 
- Year Built is 0 or Year Built is on or Before 2005

In [56]:
Softparcels2024 = arcpy.management.SelectLayerByAttribute(
    "TADParcel2024LandUse",
    "NEW_SELECTION",
    (
        "ILR <= 0.5 AND "
        "LandCode_2 IN ("
            "'Vacant - Under Development (14)', "
            "'Vacant-Developable (2)', "
            "'Parking (13)'"
        ") AND "
        "Zoning NOT IN ("
            "'Conservation District Overlay - Residential Single-Family 7.2', "
            "'Planned Development', "
            "'Planned Development - Airport Overlay - General Commercial', "
            "'Planned Development - Civic Area', "
            "'Planned Development - Community Commercial', "
            "'Planned Development - Entertainment District Overlay - Community Commercial', "
            "'Planned Development - Entertainment District Overlay - Regional Mixed-Use', "
            "'Planned Development - General Commercial', "
            "'Planned Development - Industrial Manufacturing', "
            "'Planned Development - Light Industrial', "
            "'Planned Development - Office Commercial', "
            "'Planned Development - Residential Medium Density', "
            "'Planned Development - Residential Medium Density - Community Commercial', "
            "'Planned Development - Residential Medium Density - Neighborhood Commercial', "
            "'Planned Development - Residential Medium Density - Residential Multi-Family 22', "
            "'Planned Development - Residential Multi-Family 22', "
            "'Planned Development - Residential Multi-Family 22 - Community Commercial', "
            "'Planned Development - Residential Multi-Family 22 - General Commercial', "
            "'Planned Development - Residential Single-Family 15', "
            "'Planned Development - Residential Single-Family 5', "
            "'Planned Development - Residential Single-Family 5 - Community Commercial', "
            "'Planned Development - Residential Single-Family 7.2', "
            "'Planned Development - Residential Single-Family 7.2 - Residential Medium Density - Community Commercial', "
            "'Planned Development (PD07-5R4-DP14)', "
            "'Planned Development (PD07-5R4-DP16)'"
        ") AND "
       "(Year <= 2005) AND "
        "DensityCompatible = 'Yes'"
    ),
    None
)

In [57]:
# Softparcels2024 = arcpy.management.SelectLayerByAttribute(
#     "TADParcel2024LandUse",
#     "NEW_SELECTION",
#     (
#         "ILR <= 0.5 AND "
#         "LandCode_2 IN ("
#             "'Vacant - Under Development (14)', "
#             "'Vacant-Developable (2)', "
#             "'Parking (13)'"
#         ") AND "
#         "Zoning NOT IN ("
#             "'Conservation District Overlay - Residential Single-Family 7.2'"
#         ") AND "
#         "(Year <= 2005) AND "
#         "DensityCompatible = 'Yes'"
#     ),
#     None
# )

In [58]:
softparcels = int(arcpy.management.GetCount(Softparcels2024)[0])
print(softparcels)

524


In [59]:
for row in arcpy.ListFields('TADSoftParcels2024'):
    print(row.name)

OBJECTID
Shape
Join_Count
TARGET_FID
TaxParcelI
LandCode
LandCodeDe
LandCode_1
LandCode_2
ChangeFlag
ZONINGDETA
DISTRICT
ZONINGDESC
Zoning
DensityCompatible
TAXPIN
ACRES
CALCULATED
RP
County
City
Land_Value
Improvement_Value
Total_Value
Year_Built
Land_Acres
Land_SqFt
Year
IMPvalue
LandValue
ILR
Acreage
Shape_Length
Shape_Area


In [60]:
fieldmapping = arcpy.FieldMappings()
fieldmapping.addTable(Softparcels2024)

analysisfields = ['TAXPIN', 'RP', 'LandValue', 'Improvement_Value', 'Total_Value','Year',
                  'LandCode_2','LandCodeDe','ZONINGDETA','Acreage','IMPvalue','LandValue','ILR',
                  'DISTRICT','ZONINGDESC','Zoning','DensityCompatible']

for field in fieldmapping.fields:
    if field.name not in analysisfields:
        fieldmapping.removeFieldMap(fieldmapping.findFieldMapIndex(field.name))
        

if arcpy.Exists('TADSoftParcels2024'):
    arcpy.Delete_management('TADSoftParcels2024')

arcpy.conversion.ExportFeatures(Softparcels2024,"TADSoftParcels2024","","NOT_USE_ALIAS",fieldmapping,None)
        

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcels2024'>

In [61]:

recordscount = int(arcpy.management.GetCount("TADSoftParcels2024")[0])
print(recordscount)

524


# Clusters 2024



- Create Clusters
- Calculate acreage
- Filter for clusters less than 50 acres

In [62]:
if arcpy.Exists('TADSoftParcelsClusters2024'):
    arcpy.Delete_management('TADSoftParcelsClusters2024')
    
if arcpy.Exists('TADSoftParcelsClustersTable2024'):
    arcpy.Delete_management('TADSoftParcelsClustersTable2024')


arcpy.cartography.AggregatePolygons("TADSoftParcels2024",
                                    'TADSoftParcelsClusters2024',
                                    "500 Feet",
                                    "0 SquareFeetUS",
                                    "0 SquareFeetUS",
                                    "NON_ORTHOGONAL",
                                    None,
                                    'TADSoftParcelsClustersTable2024'
                                    ,None)


<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcelsClusters2024'>

In [63]:
recordscount = int(arcpy.management.GetCount("TADSoftParcelsClusters2024")[0])
print(recordscount)

200


In [64]:
arcpy.management.AddField("TADSoftParcelsClusters2024","clusterAcreage","DOUBLE",None,None,None,"clusterAcreage","NULLABLE",
                          "NON_REQUIRED","")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcelsClusters2024'>

In [65]:
arcpy.management.CalculateGeometryAttributes("TADSoftParcelsClusters2024","clusterAcreage AREA","","ACRES",
                                             arcpy.SpatialReference(102738),"SAME_AS_INPUT")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcelsClusters2024'>

In [66]:
if arcpy.Exists('SoftParcels2024Circles'):
    arcpy.Delete_management('SoftParcels2024Circles')

arcpy.management.MinimumBoundingGeometry("TADSoftParcelsClusters2024",
                                         "SoftParcels2024Circles",
                                         "CIRCLE",
                                         "NONE",
                                         None,
                                         "NO_MBG_FIELDS")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\SoftParcels2024Circles'>

In [67]:
recordscount = int(arcpy.management.GetCount("SoftParcels2024Circles")[0])
print(recordscount)

200


# Density Based Clustering 2024

- Convert SoftParcels to Points
- Run DBSCAN On Points 
- Adjust DBSCAN for 1000 feet

In [68]:
if arcpy.Exists('TADSoftParcelsPoints2024'):
    arcpy.Delete_management('TADSoftParcelsPoints2024')

arcpy.management.FeatureToPoint("TADSoftParcels2024","TADSoftParcelsPoints2024","CENTROID")

<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftParcelsPoints2024'>

In [69]:
if arcpy.Exists('TADSoftparcelsDensityBasedClusterDistance2024'):
    arcpy.Delete_management('TADSoftparcelsDensityBasedClusterDistance2024')

arcpy.stats.DensityBasedClustering("TADSoftParcelsPoints2024",
                                   "TADSoftparcelsDensityBasedClusterDistance2024",
                                   "DBSCAN",
                                   2,
                                   "500 Feet",
                                   None,
                                   None,
                                   None)


<Result 'C:\\\\Users\\\\Thepr\\\\Documents\\\\Consulting\\\\Soft Parcel Analysis\\\\Soft Parcel Analysis.gdb\\TADSoftparcelsDensityBasedClusterDistance2024'>

In [70]:
clustersDB2024 = arcpy.management.SelectLayerByAttribute("TADSoftparcelsDensityBasedClusterDistance2024",
                                        "NEW_SELECTION",
                                        "COLOR_ID > 0",
                                        None)

recordscount = int(arcpy.management.GetCount(clustersDB2024)[0])
print(recordscount)

360


In [71]:
if arcpy.Exists('SoftParcelsPreferred2024'):
    arcpy.Delete_management('SoftParcelsPreferred2024')

arcpy.analysis.SpatialJoin("TADSoftParcels2024",clustersDB2024,"SoftParcelsPreferred2024",
                                        "JOIN_ONE_TO_ONE","KEEP_COMMON",'',"COMPLETELY_CONTAINS",None,"",None)

recordscount = int(arcpy.management.GetCount("SoftParcelsPreferred2024")[0])
print(recordscount)

360


In [72]:
if arcpy.Exists('SoftParcelsBoundary2024'):
    arcpy.Delete_management('SoftParcelsBoundary2024')

arcpy.management.MinimumBoundingGeometry("SoftParcelsPreferred2024","SoftParcelsBoundary2024","CIRCLE","LIST",
                                         "CLUSTER_ID;COLOR_ID","NO_MBG_FIELDS")

recordscount = int(arcpy.management.GetCount("SoftParcelsBoundary2024")[0])
print(recordscount)

99
